In [18]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import numpy as np
from nltk.collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
# read the job links file
job_links = pd.read_csv('/content/indeed_us_job_link_withdup.csv')

job_links.head(5)

,jk,job_page_url,radius,scrape_time,search_count,search_page_url,state,zipcode
0,30e2bbfcc708dbc3,https://www.indeed.com/viewjob?jk=30e2bbfcc708...,7,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=27839&radius=7&l...,NC,27839.0
1,bc37c28ff65bf344,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,3,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=05871&radius=3&l...,VT,5871.0
2,6511d4c0145d8941,https://www.indeed.com/viewjob?jk=6511d4c0145d...,3,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=05871&radius=3&l...,VT,5871.0
3,570586a11d074703,https://www.indeed.com/viewjob?jk=570586a11d07...,3,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=05871&radius=3&l...,VT,5871.0
4,e94e4e724bb36223,https://www.indeed.com/viewjob?jk=e94e4e724bb3...,3,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=05871&radius=3&l...,VT,5871.0


In [20]:
# read the job descriptions file
job_descs = pd.read_csv('/content/indeed_us_job_description 2.csv', error_bad_lines = False, engine = 'python')

job_descs.head(5)



<ipython-input-20-10e6d6bebb0b>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  job_descs = pd.read_csv('/content/indeed_us_job_description 2.csv', error_bad_lines = False, engine = 'python')
Skipping line 1450: unexpected end of data


,benefits,company,insights,jk,job_date,job_description,job_details,job_label,job_page_url,job_title,location,qualifications,rating,salary,scrape_time,state,zipcode,estimated_salary
0,BenefitsPulled from the full job description ...,Justin Rosen F D S Inc,Hiring Insights Hiring 1 candidate for this ro...,6516a3814e551ad1,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...","$44,224 - $46,722 a year - Full-time",NaN,https://www.indeed.com/viewjob?jk=6516a3814e55...,Bookkeeper,"<div>Tampa, FL 33629</div>",QualificationsBookkeeping: 1 year (Required),NaN,"$44,224 - $46,722 a year",2022-10-12 09:41:41,FL,33609,NaN
1,NaN,Amici | Amici Food Group,Hiring Insights Job activity Posted 6 days ago,ee2f9c94619d0fbe,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...",Full-time,NaN,https://www.indeed.com/viewjob?jk=ee2f9c94619d...,Dishwasher,"<div>Madison, GA 30650</div>",NaN,NaN,Full-time,2022-10-12 09:41:41,GA,30667,$19.9K - $25.3K
2,NaN,Ballpark Realty,Hiring Insights Hiring 1 candidate for this ro...,5a031cbd3c84edfb,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...",$19 - $22 an hour - Full-time,NaN,https://www.indeed.com/viewjob?jk=5a031cbd3c84...,Sales and Marketing Coordinator,"<div>Snohomish, WA 98296</div>",QualificationsMarketing: 1 year (Required)Work...,NaN,$19 - $22 an hour,2022-10-12 09:41:41,WA,98296,NaN
3,NaN,Rural Community Transportation,Hiring Insights Job activity Posted 6 days ago,bc37c28ff65bf344,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...","$50,000 a year - Full-time",NaN,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,Procurement Specialist,"<div>Lyndonville, VT 05851</div>",NaN,NaN,"$50,000 a year",2022-10-12 09:41:41,VT,5871,NaN
4,BenefitsPulled from the full job description ...,United States Postal Service,Hiring Insights Job activity Posted 6 days ago,ec45994a00b19656,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...",Part-time,NaN,https://www.indeed.com/viewjob?jk=ec45994a00b1...,CITY CARRIER ASSISTANT,"<div>McMinnville, OR 97128</div>",NaN,NaN,Part-time,2022-10-12 09:41:42,OR,97101,NaN


In [21]:
merged_df = pd.merge(job_links, job_descs, on = ['jk', 'state', 'zipcode'])

merged_df.head(5)

,jk,job_page_url_x,radius,scrape_time_x,search_count,search_page_url,state,zipcode,benefits,company,...,job_details,job_label,job_page_url_y,job_title,location,qualifications,rating,salary,scrape_time_y,estimated_salary
0,bc37c28ff65bf344,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,3,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=05871&radius=3&l...,VT,5871.0,NaN,Rural Community Transportation,...,"$50,000 a year - Full-time",NaN,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,Procurement Specialist,"<div>Lyndonville, VT 05851</div>",NaN,NaN,"$50,000 a year",2022-10-12 09:41:41,NaN
1,06ebf369510b19e4,https://www.indeed.com/viewjob?jk=06ebf369510b...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Siemens,...,Full-time,NaN,https://www.indeed.com/viewjob?jk=06ebf369510b...,Inside Parts Sales Representative - National O...,"<div>Concord, NH 03303</div>",NaN,NaN,Full-time,2022-10-12 09:41:44,NaN
2,e688e1ae76f057f5,https://www.indeed.com/viewjob?jk=e688e1ae76f0...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Host Healthcare,...,NaN,NaN,https://www.indeed.com/viewjob?jk=e688e1ae76f0...,RN - Medical-Surgical,"<div>Lebanon, NH</div>",NaN,NaN,NaN,2022-10-12 09:41:45,$76.8K - $97.2K
3,cb1a27f158105ec1,https://www.indeed.com/viewjob?jk=cb1a27f15810...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Gifted Healthcare,...,Travel nursing,NaN,https://www.indeed.com/viewjob?jk=cb1a27f15810...,Registered Nurse - Psychiatric RN Travel,"<div>Laconia, NH</div>",NaN,NaN,Travel nursing,2022-10-12 09:41:46,NaN
4,38b2541dd2d8ae0d,https://www.indeed.com/viewjob?jk=38b2541dd2d8...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Upper Valley Haven,...,$18 - $22 an hour - Full-time,NaN,https://www.indeed.com/viewjob?jk=38b2541dd2d8...,Food Shelf & Front End Associate,"<div>White River Junction, VT 05001</div>",NaN,NaN,$18 - $22 an hour,2022-10-12 09:41:47,NaN


In [22]:
def clean_description(text):
    
    if text is None:
        # if the input is NaN or None, return an empty string
        return ''
    text = re.sub(r'<.*?>', '', text).replace('\n', '') #clean the html and newline characters
    text = re.sub(r'[^a-zA-Z0-9 \n\.]', '', text) # clean all special characters include punctuations
    text = text.lower() # convert the lower 
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
    
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)

    stemmer = PorterStemmer()

    # remove the stopwords and stem the words by using PorterStemmer
    clean_text = []
    for word in word_tokens:
      if word not in stop_words:
        if len(word) > 1:
          clean_word = stemmer.stem(word)
          clean_text.append(clean_word)

    return " ".join(clean_text)
      
    

In [23]:
# O(n) = O(longn + k) k -> number of unique group
merged_df['job_description'] = merged_df['job_description'].apply(lambda x: clean_description(x))

#assign the job_description at the end of dataframe
merged_df = merged_df.assign(job_description=merged_df.pop('job_description'))
merged_df.head(5)

,jk,job_page_url_x,radius,scrape_time_x,search_count,search_page_url,state,zipcode,benefits,company,...,job_label,job_page_url_y,job_title,location,qualifications,rating,salary,scrape_time_y,estimated_salary,job_description
0,bc37c28ff65bf344,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,3,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=05871&radius=3&l...,VT,5871.0,NaN,Rural Community Transportation,...,NaN,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,Procurement Specialist,"<div>Lyndonville, VT 05851</div>",NaN,NaN,"$50,000 a year",2022-10-12 09:41:41,NaN,procur specialist respons procur activ process...
1,06ebf369510b19e4,https://www.indeed.com/viewjob?jk=06ebf369510b...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Siemens,...,NaN,https://www.indeed.com/viewjob?jk=06ebf369510b...,Inside Parts Sales Representative - National O...,"<div>Concord, NH 03303</div>",NaN,NaN,Full-time,2022-10-12 09:41:44,NaN,insid part sale repres nation oper center noc ...
2,e688e1ae76f057f5,https://www.indeed.com/viewjob?jk=e688e1ae76f0...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Host Healthcare,...,NaN,https://www.indeed.com/viewjob?jk=e688e1ae76f0...,RN - Medical-Surgical,"<div>Lebanon, NH</div>",NaN,NaN,NaN,2022-10-12 09:41:45,$76.8K - $97.2K,job host healthcar awardwin travel healthcar c...
3,cb1a27f158105ec1,https://www.indeed.com/viewjob?jk=cb1a27f15810...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Gifted Healthcare,...,NaN,https://www.indeed.com/viewjob?jk=cb1a27f15810...,Registered Nurse - Psychiatric RN Travel,"<div>Laconia, NH</div>",NaN,NaN,Travel nursing,2022-10-12 09:41:46,NaN,gift healthcar experi gift healthcar awardwin ...
4,38b2541dd2d8ae0d,https://www.indeed.com/viewjob?jk=38b2541dd2d8...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Upper Valley Haven,...,NaN,https://www.indeed.com/viewjob?jk=38b2541dd2d8...,Food Shelf & Front End Associate,"<div>White River Junction, VT 05001</div>",NaN,NaN,$18 - $22 an hour,2022-10-12 09:41:47,NaN,upper valley provid temporari shelter support ...


In [24]:
group_df= merged_df.groupby(['job_title', 'company', 'job_description'])['jk'].apply(lambda x :list(np.unique(x))).reset_index(name = 'jk_list')

merged_df['jk_list'] = group_df['jk_list']

#merged_df.sample(frac=1).head(10)

merged_df['jk_list'].head(50)

0                       [f23c59279305add7]
1                       [1fd3cea2aa6a17f7]
2                       [3d1825f1f4136389]
3                       [51ace95c480c0eb1]
4     [9054265a61836546, e2e7b86804516402]
5                       [cb41f70397a60b7d]
6                       [a628fc2196cd7e87]
7                       [ca8384fcfcf0ef5f]
8                       [e9fe5f6dba94a06e]
9                       [2cf0c5f24dbe52ff]
10                      [e75ab65075eeaf05]
11                      [095ba425c295daec]
12                      [7886f20820f3bd6d]
13                      [052e4c146b607c7a]
14                      [f74dab2dc29a0d2b]
15                      [c9ad8baebc54a449]
16                      [a1eec5866b5aa632]
17                      [83a1d572dce6d420]
18                      [bfa8b225d193ccf5]
19                      [06f809c8f37daf43]
20                      [146369037a3888ec]
21                      [5c09e270636d4fc8]
22                      [2e558cdc8c7589e9]
23         

In [25]:
#time complexity is O(n)
deduped_df = merged_df.drop_duplicates(subset=['job_description', 'job_title', 'company'], keep='first')

deduped_df.head(5)

,jk,job_page_url_x,radius,scrape_time_x,search_count,search_page_url,state,zipcode,benefits,company,...,job_page_url_y,job_title,location,qualifications,rating,salary,scrape_time_y,estimated_salary,job_description,jk_list
0,bc37c28ff65bf344,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,3,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=05871&radius=3&l...,VT,5871.0,NaN,Rural Community Transportation,...,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,Procurement Specialist,"<div>Lyndonville, VT 05851</div>",NaN,NaN,"$50,000 a year",2022-10-12 09:41:41,NaN,procur specialist respons procur activ process...,[f23c59279305add7]
1,06ebf369510b19e4,https://www.indeed.com/viewjob?jk=06ebf369510b...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Siemens,...,https://www.indeed.com/viewjob?jk=06ebf369510b...,Inside Parts Sales Representative - National O...,"<div>Concord, NH 03303</div>",NaN,NaN,Full-time,2022-10-12 09:41:44,NaN,insid part sale repres nation oper center noc ...,[1fd3cea2aa6a17f7]
2,e688e1ae76f057f5,https://www.indeed.com/viewjob?jk=e688e1ae76f0...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Host Healthcare,...,https://www.indeed.com/viewjob?jk=e688e1ae76f0...,RN - Medical-Surgical,"<div>Lebanon, NH</div>",NaN,NaN,NaN,2022-10-12 09:41:45,$76.8K - $97.2K,job host healthcar awardwin travel healthcar c...,[3d1825f1f4136389]
3,cb1a27f158105ec1,https://www.indeed.com/viewjob?jk=cb1a27f15810...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Gifted Healthcare,...,https://www.indeed.com/viewjob?jk=cb1a27f15810...,Registered Nurse - Psychiatric RN Travel,"<div>Laconia, NH</div>",NaN,NaN,Travel nursing,2022-10-12 09:41:46,NaN,gift healthcar experi gift healthcar awardwin ...,[51ace95c480c0eb1]
4,38b2541dd2d8ae0d,https://www.indeed.com/viewjob?jk=38b2541dd2d8...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Upper Valley Haven,...,https://www.indeed.com/viewjob?jk=38b2541dd2d8...,Food Shelf & Front End Associate,"<div>White River Junction, VT 05001</div>",NaN,NaN,$18 - $22 an hour,2022-10-12 09:41:47,NaN,upper valley provid temporari shelter support ...,"[9054265a61836546, e2e7b86804516402]"


In [26]:
deduped_df = deduped_df.reset_index(drop=True)
deduped_df['index_dup'] = deduped_df.index + 1

In [27]:
#change the jk_list and index_dump column
cols = deduped_df.columns.tolist()

# reposition last two columns
cols = cols[:-2] + cols[-2:][::-1]

# reindex dataframe with new column order
deduped_df = deduped_df.reindex(columns=cols)


In [28]:
deduped_df.head(5)

,jk,job_page_url_x,radius,scrape_time_x,search_count,search_page_url,state,zipcode,benefits,company,...,job_title,location,qualifications,rating,salary,scrape_time_y,estimated_salary,job_description,index_dup,jk_list
0,bc37c28ff65bf344,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,3,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=05871&radius=3&l...,VT,5871.0,NaN,Rural Community Transportation,...,Procurement Specialist,"<div>Lyndonville, VT 05851</div>",NaN,NaN,"$50,000 a year",2022-10-12 09:41:41,NaN,procur specialist respons procur activ process...,1,[f23c59279305add7]
1,06ebf369510b19e4,https://www.indeed.com/viewjob?jk=06ebf369510b...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Siemens,...,Inside Parts Sales Representative - National O...,"<div>Concord, NH 03303</div>",NaN,NaN,Full-time,2022-10-12 09:41:44,NaN,insid part sale repres nation oper center noc ...,2,[1fd3cea2aa6a17f7]
2,e688e1ae76f057f5,https://www.indeed.com/viewjob?jk=e688e1ae76f0...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Host Healthcare,...,RN - Medical-Surgical,"<div>Lebanon, NH</div>",NaN,NaN,NaN,2022-10-12 09:41:45,$76.8K - $97.2K,job host healthcar awardwin travel healthcar c...,3,[3d1825f1f4136389]
3,cb1a27f158105ec1,https://www.indeed.com/viewjob?jk=cb1a27f15810...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Gifted Healthcare,...,Registered Nurse - Psychiatric RN Travel,"<div>Laconia, NH</div>",NaN,NaN,Travel nursing,2022-10-12 09:41:46,NaN,gift healthcar experi gift healthcar awardwin ...,4,[51ace95c480c0eb1]
4,38b2541dd2d8ae0d,https://www.indeed.com/viewjob?jk=38b2541dd2d8...,6,2022-10-06 01:00:19,NaN,https://www.indeed.com/jobs?l=03230&radius=6&l...,NH,3230.0,BenefitsPulled from the full job description ...,Upper Valley Haven,...,Food Shelf & Front End Associate,"<div>White River Junction, VT 05001</div>",NaN,NaN,$18 - $22 an hour,2022-10-12 09:41:47,NaN,upper valley provid temporari shelter support ...,5,"[9054265a61836546, e2e7b86804516402]"


In [29]:
deduped_df.to_csv('final.csv', index=False)